In [1]:
#import dependencies
from bs4 import BeautifulSoup
from datetime import datetime
import requests 
import pandas as pd
import os

In [53]:
def retreive_stats_page_text():
    stats_page = requests.get("https://pokemonshowdown.com/users/chillidawg")
    stats_page_soup = BeautifulSoup(stats_page.content, "html.parser")
    stats_page_text = stats_page_soup.text
    return stats_page_text

def obtain_randbats_elo(page_text):
    ladder_index = page_text.find("gen9randombattle")
    start_number_index = ladder_index + len("gen9randombattle")
    end_number_index = start_number_index + 4
    return page_text[start_number_index: end_number_index]

#add a check to see if elo has changed

def obtain_formatted_date():
    now = datetime.now()
    return now.strftime("%d/%m/%Y %H:%M:%S")

def find(name, path):
    """finds the csv file"""
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root,name) 
        else:
            return None

def update_csv():
    """if csv file present, change into pandas dataframe before updating. if no csv file present, make one"""

    new_row = {'datetime': obtain_formatted_date(), 'Elo': obtain_randbats_elo(retreive_stats_page_text())}
    randbats_elos_csv = find('randbats_elo.csv', '.')

    if randbats_elos_csv == None:
        randbats_elos = pd.DataFrame(new_row, index=[0])
    else:
        randbats_elos = pd.read_csv(randbats_elos_csv)
        print(randbats_elos, new_row)

        randbats_elos = randbats_elos.append(new_row, ignore_index = True)
        print(randbats_elos)

    randbats_elos.to_csv('randbats_elo.csv', index=False)

In [55]:
update_csv()

              datetime   Elo
0  09/11/2023 16:36:50  1831
1  09/11/2023 16:41:19  1831 {'datetime': '09/11/2023 16:56:57', 'Elo': '1831'}
              datetime   Elo
0  09/11/2023 16:36:50  1831
1  09/11/2023 16:41:19  1831
2  09/11/2023 16:56:57  1831


/var/folders/1q/8vymbqss1bgdgr4kvck49gtw0000gn/T/ipykernel_1049/3846074680.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randbats_elos = randbats_elos.append(new_row, ignore_index = True)
